$$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\Yv}{\mathbf{Y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\gv}{\mathbf{g}}
\newcommand{\Hv}{\mathbf{H}}
\newcommand{\dv}{\mathbf{d}}
\newcommand{\Vv}{\mathbf{V}}
\newcommand{\vv}{\mathbf{v}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\Sv}{\mathbf{S}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\Zv}{\mathbf{Z}}
\newcommand{\Norm}{\mathcal{N}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}$$

# Introduction to Classification

The primary difference between regression and classification is the type of output.  In regression, the output is one or more continuous values.  In classification, the outputs are class labels such as "default" or "no default".

## Logistic Regression

**Binary classification** is the simpliest form of classification.  In this type of problem, our job is to separate the data into one of two mutually exclusive classes.  For example, say we want to model whether a credit card holder will default on their monthly payment or not.  Our two output classes might be Y = 0 if no default and Y = 1 if default occurred.

Let's take a look at the credit card default data at the UCI site.  The original data was pulled from: [https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients), downloaded locally, converted to a csv file and then upload to my github repo.

In [19]:
# Start with the standard incantation to Python gods...
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
import csv
import pandas as pd
data_url = "https://raw.githubusercontent.com/MichaelSzczepaniak/MLToolbox/master/data/default%20of%20credit%20card%20clients.csv"
#data_url = "https://www.dropbox.com/s/9p9szqwsvi9q9gl/default%20of%20credit%20card%20clients.csv?dl=1"
cc_data = pd.read_csv(data_url, dtype={'X12' : np.float32, 'Y' : np.float32}, header=[0, 1])

In [31]:
cc_data[:5]

,Unnamed: 0_level_0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1.0
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1.0
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0.0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0.0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0.0


From the documentation, we see that X12 is the latest billing amount (September 2005) and that Y indicates whether there was a default (Y = 1) or not (y = 0).  Let's start with a quick plot of these two variables.

In [39]:
x = cc_data['X12']
print(x[1:2])
type(x)
#x = x.as_matrix()
#y = cc_data['Y']
#x = y.as_matrix()
#plt.plot(x, y)

   BILL_AMT1
1     2682.0


pandas.core.frame.DataFrame

Instead of directly modeling which class a sample belongs to, logistic regression starts by modeling the probability $p(Y=1\:|\:\Xv)$.  In other words, we are modelling the probability that an n-dimensional sample $\Xv$ will be in the class $Y = 1$.